# Geolocalización con MultiProcessing + IGR Palabras

En esta notebook haremos un intento de geolocalización con los textos de los usuarios...

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_json("../data/geoloc/users_train.json")
df_test = pd.read_json("../data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [6]:
df_train.groupby("provincia").count()

,text
provincia,
buenosaires,337
catamarca,341
chaco,331
chubut,328
cordoba,317
corrientes,345
entrerios,338
formosa,286
jujuy,339


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [7]:
%%time
from contrastes.processing import build_dataframe_from_users
from contrastes.processing import preprocess_raw_df


#word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

word_df = pd.read_csv("train_word_df_filtered.csv", index_col=0)
word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

CPU times: user 680 ms, sys: 80 ms, total: 760 ms
Wall time: 762 ms


/home/jmperez/projects/contrastes/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


La reg. logística será un softmax, así que elijo `multi_class='multinomial'`

In [8]:
from contrastes.igr import igr

word_df["igr"] = igr(word_df, word_df.columnas_palabras)

In [9]:
word_df.sort_values("igr", ascending=False, inplace=True)

word_df[:30][word_df.columnas_palabras]

,buenosaires_ocurrencias,catamarca_ocurrencias,chaco_ocurrencias,chubut_ocurrencias,cordoba_ocurrencias,corrientes_ocurrencias,entrerios_ocurrencias,formosa_ocurrencias,jujuy_ocurrencias,lapampa_ocurrencias,...,neuquen_ocurrencias,rionegro_ocurrencias,salta_ocurrencias,sanjuan_ocurrencias,sanluis_ocurrencias,santacruz_ocurrencias,santafe_ocurrencias,santiago_ocurrencias,tierradelfuego_ocurrencias,tucuman_ocurrencias
rioja,21.0,219.0,3.0,18.0,23.0,21.0,6.0,2.0,27.0,9.0,...,4.0,7.0,57.0,88.0,3.0,10.0,45.0,11.0,4.0,44.0
logroño,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hoa,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3275.0,...,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
ushuaia,19.0,2.0,3.0,16.0,3.0,2.0,3.0,4.0,6.0,7.0,...,7.0,13.0,72.0,8.0,4.0,35.0,6.0,14.0,5640.0,2.0
chepes,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
tiemposur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,883.0,0.0,0.0,0.0,0.0
palpala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,591.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
beder,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
quitilipi,0.0,0.0,1043.0,0.0,0.0,10.0,0.0,3.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oberá,0.0,1.0,1.0,3.0,0.0,2.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [10]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from contrastes.text import tokenize

liw_vectorizer = CountVectorizer(
    tokenizer=tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 10min 1s, sys: 580 ms, total: 10min 2s
Wall time: 10min 2s


Ya las tenemos vectorizadas en el orden esperado!

In [11]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

In [12]:
%%time
from contrastes.classifiers import fit_classifiers

num_words_to_fit = list(range(250, 5000, 250)) + list(range(5000, 20000, 500))

params = {"max_iter": 7000}

ret = fit_classifiers(X_train, y_train, X_test, y_test, 
                      province_encoder=province_encoder, clf_params=params,
                      range_num_words=num_words_to_fit, num_jobs=8)

Classifier params: {'multi_class': 'multinomial', 'solver': 'saga', 'penalty': 'l2', 'max_iter': 7000}
Entrenando con 250 palabras
Entrenando con 750 palabras
Entrenando con 1750 palabras
Entrenando con 1250 palabras
Entrenando con 2250 palabras
Entrenando con 3250 palabras
Entrenando con 3750 palabras
Entrenando con 2750 palabras
250   palabras ----> accuracy 42.92 mean distance 588.8424
Entrenando con 500 palabras
750   palabras ----> accuracy 58.24 mean distance 424.144
Entrenando con 1000 palabras
1250  palabras ----> accuracy 63.28 mean distance 320.6816
Entrenando con 1500 palabras
1750  palabras ----> accuracy 66.36 mean distance 264.9508
Entrenando con 2000 palabras
2250  palabras ----> accuracy 67.52 mean distance 258.6624
Entrenando con 2500 palabras
2750  palabras ----> accuracy 70.16 mean distance 228.6856
Entrenando con 3000 palabras
3250  palabras ----> accuracy 70.68 mean distance 233.0544
Entrenando con 3500 palabras
3750  palabras ----> accuracy 71.84 mean distance 223

In [13]:
for r in ret:
    num_words = r["num_words"]
    acc = r["accuracy"]
    md = r["mean_distance"]
    print("{:<5} palabras ----> accuracy {:.2f} mean distance {}".format(
        num_words, acc*100, md
    ))

250   palabras ----> accuracy 42.92 mean distance 588.8424
500   palabras ----> accuracy 52.28 mean distance 495.398
750   palabras ----> accuracy 58.24 mean distance 424.144
1000  palabras ----> accuracy 61.96 mean distance 330.23
1250  palabras ----> accuracy 63.28 mean distance 320.6816
1500  palabras ----> accuracy 65.00 mean distance 275.7196
1750  palabras ----> accuracy 66.36 mean distance 264.9508
2000  palabras ----> accuracy 67.44 mean distance 257.8484
2250  palabras ----> accuracy 67.52 mean distance 258.6624
2500  palabras ----> accuracy 69.32 mean distance 236.1408
2750  palabras ----> accuracy 70.16 mean distance 228.6856
3000  palabras ----> accuracy 70.44 mean distance 230.708
3250  palabras ----> accuracy 70.68 mean distance 233.0544
3500  palabras ----> accuracy 71.00 mean distance 231.2224
3750  palabras ----> accuracy 71.84 mean distance 223.812
4000  palabras ----> accuracy 72.56 mean distance 219.6388
4250  palabras ----> accuracy 73.56 mean distance 214.0228
450

In [16]:
import pickle

pickle.dump(ret, open("res_igr_palabras.pkl", "wb"))

In [17]:
new_ret = pickle.load(open("res_igr_palabras.pkl", "rb"))

clf = new_ret[1]["clf"]

clf.coef_.shape

(23, 500)